In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time

# 크롬 드라이버 세팅
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://patent.scourt.go.kr/dcboard/new/DcNewsListAction.work?gubun=44&cbub_code=")

data = []

def collect_data():
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "td.title a"))
    )
    links = driver.find_elements(By.CSS_SELECTOR, "td.title a")
    for link in links:
        title = link.text.strip()
        href = link.get_attribute("href")
        data.append({"제목": title, "링크": href})

while True:
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.pagination li a"))
        )

        # 페이지 텍스트 수집
        page_links = driver.find_elements(By.CSS_SELECTOR, "ul.pagination li a")
        page_texts = []
        for el in page_links:
            text = el.text.strip()
            if text.isdigit():
                page_texts.append(text)

        # 각 페이지 번호 클릭하며 크롤링
        for page_num in page_texts:
            try:
                print(f"\n📄 {page_num} 페이지 크롤링 중...")

                # 페이지 클릭 시도 (새로 DOM 갱신)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//ul[@class='pagination']//a[text()='{page_num}']"))
                )
                page_el = driver.find_element(By.XPATH, f"//ul[@class='pagination']//a[text()='{page_num}']")
                driver.execute_script("arguments[0].click();", page_el)
                time.sleep(1.5)
                collect_data()
            except Exception as e:
                print(f"❌ 페이지 {page_num} 클릭 실패: {e}")
                continue

        # 다음 버튼 클릭
        next_button = None
        for el in driver.find_elements(By.CSS_SELECTOR, "ul.pagination li"):
            a_tag = el.find_element(By.TAG_NAME, "a")
            if el.text.strip() == "다음":
                #비활성화 여부 확인
                class_attr = el.get_attribute("class")
                if "disabled" in class_attr:
                    next_button = None
                else:
                    next_button = a_tag
                break

        if next_button:
            print("➡️ 다음 페이지 그룹으로 이동")
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(2)
        else:
            print("✅ 더 이상 다음 페이지 그룹 없음. 크롤링 종료.")
            break

    except TimeoutException:
        print("❌ 페이지 로딩 타임아웃")
        break
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        break

# 저장
with open("patent_cases_all_pages.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

driver.quit()
print("\n✅ JSON 저장 완료! 'patent_cases_all_pages.json' 파일 생성됨.")

ModuleNotFoundError: No module named 'selenium'